In [1]:
# Primary modules
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# Other modules / functions
import pickle
import os
import matplotlib.gridspec as gridspec

# For inline plotting
%matplotlib inline
plt.ion()

In [2]:
# Import suftware (use development copy)
import sys
sys.path.append('../') #('../../suftware')

import suftware as sw
from supplemental_functions import gkde, dpmm, KL_divergence, p_value_cal

In [3]:
# Specify simulation parameters
num_simulations = 3
Ns = [10, 100]

# Specify DEFT parameters
alphas = [1, 2, 3, 4]
num_posterior_samples = 100
t_range = 10

# Save result ?
save_result = True
dir_result = './simulations_test/fig2_figS3/'
result_cols = ['DEFT1_10', 'DEFT2_10', 'DEFT3_10', 'DEFT4_10', 'GKDE_10', 'DPMM_10',
               'DEFT1_100','DEFT2_100','DEFT3_100','DEFT4_100','GKDE_100','DPMM_100']

In [4]:
distribution_names = [
     'DoubleGaussian',
     'Pareto',
     'Argus',
     'Beta',
     'Bradford',
     'Cauchy',
     'ChiSquare',
     'ExpPower',
     'ExpWeibull',
     'FoldedNormal',
     'SkewNorm',
     'Wald',
     'WeibullMin',
     'WrapCauchy']
    
for i, distribution_name in enumerate(distribution_names):
    print('Simulating %s...'%distribution_name)
    print()
    
    dist = sw.SimulatedDensity(name=distribution_name)
    xs = np.linspace(dist.xmin, dist.xmax, 100)
    dx = xs[1] - xs[0]
    ys = dist.evaluate(xs)
    Q_true = ys / np.sum(ys*dx)
    
    #
    # Start iteration
    #

    KL_divergences = np.zeros([num_simulations, len(Ns)*(len(alphas)+2)])
    p_values = np.zeros([num_simulations, len(Ns)*(len(alphas)+2)])
    
    i = 0
    while i < num_simulations:

        KL_divergences_i = []
        p_values_i = []

        for N in Ns:

            print('N =', N)

            #
            # Generate data
            #

            zs = np.linspace(dist.xmin, dist.xmax, 10001)
            dz = zs[1] - zs[0]
            zs = np.linspace(dist.xmin+dz/2, dist.xmax-dz/2, 10000)
            probs = dist.evaluate(zs) / np.sum(dist.evaluate(zs))
            data = np.random.choice(zs, size=N, replace=True, p=probs)
            jitter = dz * (np.random.rand(N)-0.5)
            data += jitter
            
            #
            # DEFT
            #

            for alpha in alphas:

                print('DEFT, alpha =', alpha, '...')
                
                # Prepare t_starts and DT_MAXs (The first one is default)
                t_start = min(0.0, np.log(N)-2.0*alpha*np.log(alpha/dx))
                if t_start < -10.0:
                    t_start /= 2
                t_starts = [None] + list(t_start+np.linspace(1,t_range,t_range)) \
                                  + list(t_start-np.linspace(1,t_range,t_range))
                DT_MAXs = [1.0, 0.5, 0.25, 0.1]
                
                # Run DEFT
                for t_start in t_starts:    
                    for DT_MAX in DT_MAXs:                    
                        try:
                            density = sw.DensityEstimator(data, grid=xs, alpha=alpha, 
                                                          num_posterior_samples=num_posterior_samples, 
                                                          t_start=t_start, max_t_step=DT_MAX)
                            ERROR_switch = False
                            break
                        except:
                            ERROR_switch = True
                            continue
                    if ERROR_switch == False:
                        break
                    if ERROR_switch == True:
                        continue
    
                # If DEFT succeeded, proceed to calculate other stuff
                if ERROR_switch == False:
                    Q_deft = density.values
                    Q_deft_samples = density.evaluate_samples(xs)

                    # Calculate D(Q_true||Q_deft)
                    KL_divergence_deft = KL_divergence(Q_true, Q_deft, dx)
                    KL_divergences_i.append(KL_divergence_deft)

                    # Calculate D(Q_deft_samples||Q_deft)
                    KL_divergences_deft = np.zeros(num_posterior_samples)
                    for k in range(num_posterior_samples):
                        KL_divergences_deft[k] = KL_divergence(Q_deft_samples[:,k], Q_deft, dx)

                    # Compute p-value
                    p_value_deft = p_value_cal(KL_divergences_deft, KL_divergence_deft)
                    p_values_i.append(p_value_deft)

                # If DEFT failed, break the loop
                if ERROR_switch == True:
                    break
                
            if ERROR_switch == True:
                break  
                
            #
            # GKDE
            #

            print('GKDE ...')

            # Run GKDE
            Q_gkde, Q_gkde_samples, ERROR_switch = gkde(data, xs, num_posterior_samples)
            if ERROR_switch == True:
                break

            # Calculate D(Q_true||Q_gkde)
            KL_divergence_gkde = KL_divergence(Q_true, Q_gkde, dx)
            KL_divergences_i.append(KL_divergence_gkde)

            # Calculate D(Q_gkde_samples||Q_gkde)
            KL_divergences_gkde = np.zeros(num_posterior_samples)
            for k in range(num_posterior_samples):
                KL_divergences_gkde[k] = KL_divergence(Q_gkde_samples[:,k], Q_gkde, dx)

            # Compute p-value
            p_value_gkde = p_value_cal(KL_divergences_gkde, KL_divergence_gkde)
            p_values_i.append(p_value_gkde)
            
            #
            # DPMM
            #

            print('DPMM ...')

            # Run DPMM
            Q_dpmm, Q_dpmm_samples, ERROR_switch = dpmm(data, xs, num_posterior_samples)
            if ERROR_switch == True:
                break

            # Calculate D(Q_true||Q_dpmm)
            KL_divergence_dpmm = KL_divergence(Q_true, Q_dpmm, dx)        
            KL_divergences_i.append(KL_divergence_dpmm)

            # Calculate D(Q_dpmm_samples||Q_dpmm)
            KL_divergences_dpmm = np.zeros(num_posterior_samples)
            for k in range(num_posterior_samples):
                KL_divergences_dpmm[k] = KL_divergence(Q_dpmm_samples[:,k], Q_dpmm, dx)

            # Compute p-value
            p_value_dpmm = p_value_cal(KL_divergences_dpmm, KL_divergence_dpmm)
            p_values_i.append(p_value_dpmm)
            
        #
        # Proceed
        #
        
        if ERROR_switch == False:
            print('%s simulation   # %s' % (distribution_name,i))
            print()
            KL_divergences[i,:] = np.array(KL_divergences_i)
            p_values[i,:] = np.array(p_values_i)
            i += 1
            
    #
    # Save result
    #
    
    if save_result:
        pd.DataFrame(data=KL_divergences, columns=result_cols) \
            .to_pickle(dir_result+'df_KL_divergences_'+dist.name+'.pkl')
        pd.DataFrame(data=p_values, columns=result_cols) \
            .to_pickle(dir_result+'df_p_values_'+dist.name+'.pkl')
    
print('--- done ---')
print()

Simulating DoubleGaussian...

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
DoubleGaussian simulation   # 0

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
DoubleGaussian simulation   # 1

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
DoubleGaussian simulation   # 2

Simulating Pareto...

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
Paret

DPMM ...
SkewNorm simulation   # 2

Simulating Wald...

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
Wald simulation   # 0

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
Wald simulation   # 1

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
Wald simulation   # 2

Simulating WeibullMin...

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
Weibu

/anaconda3/lib/python3.6/site-packages/numpy/lib/scimath.py:262: RuntimeWarning: divide by zero encountered in log
  return nx.log(x)


Error:  /log_ptgd/ log_det is not finite at t = -0.8239296420713621: log_det = -inf
Error:  /compute_corrector_step/ phi is not converging at t = -13.093201009804908: beta = 5.345529420184391e-51
Error:  /compute_corrector_step/ phi is not converging at t = -13.140098428167066: beta = 5.345529420184391e-51
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
WeibullMin simulation   # 1

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
Error:  /compute_corrector_step/ phi is not converging at t = -11.863891285524069: beta = 5.345529420184391e-51
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
WeibullMin simulation   # 2

Simulating WrapCauchy...

N = 10
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, alpha = 3 ...
DEFT, alpha = 4 ...
GKDE ...
DPMM ...
N = 100
DEFT, alpha = 1 ...
DEFT, alpha = 2 ...
DEFT, al